In [1]:
import os
from datasets import load_from_disk
from Geneformer.geneformer import Classifier, TranscriptomeTokenizer
from transformers import TrainingArguments

/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tk = TranscriptomeTokenizer(
#         custom_attr_name_dict={
#             "disease_group": "disease_group",
#         },
#         nproc=16  # adjust based on your system
#     )

# tk.tokenize_data(
#     "train_dataset",
#     "tokenized_train_dataset",
#     "tokenized",
#     file_format = "h5ad"  
# )

# tk.tokenize_data(
#     "test_dataset",
#     "tokenized_test_dataset",
#     "tokenized",
#     file_format = "h5ad"  
# )

In [3]:
print("=== RUNNING QUICK TEST WITH CORRECT API ===")
    
# Load and subset data
full_dataset = load_from_disk("tokenized_train_dataset/tokenized.dataset")
subset = full_dataset.select(range(100))  # slightly more for meaningful test
subset = subset.remove_columns('length')
subset_path = "tokenized_train_dataset/subset_test.dataset"
subset.save_to_disk(subset_path)

os.makedirs("quick_test_output", exist_ok=True)

=== RUNNING QUICK TEST WITH CORRECT API ===


In [4]:
import pandas
df = subset.to_pandas()
df

,input_ids,disease_group
0,"[2, 18875, 9387, 16079, 13985, 7126, 16450, 66...",Inactive
1,"[2, 9217, 14319, 14806, 11632, 7716, 3495, 726...",Inactive
2,"[2, 5788, 15343, 12186, 7590, 8735, 3857, 1701...",TNJDM
3,"[2, 915, 4305, 11429, 9572, 15371, 1001, 241, ...",Inactive
4,"[2, 16736, 15270, 12350, 744, 4331, 3174, 4155...",TNJDM
...,...,...
95,"[2, 5147, 11990, 17364, 11054, 13685, 5365, 15...",TNJDM
96,"[2, 10643, 10673, 6331, 5689, 15032, 3462, 162...",Inactive
97,"[2, 5301, 1815, 18787, 14578, 12978, 7042, 140...",TNJDM
98,"[2, 9404, 2649, 7878, 12978, 12668, 8025, 9789...",Inactive


In [5]:
quick_training_args = TrainingArguments(
      output_dir="quick_test_output",
      num_train_epochs=1,              # just 3 epochs for quick test
      learning_rate=5e-5,
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      warmup_steps=50,
      weight_decay=0.01,
      logging_steps=10,
      eval_strategy="no",        # no eval for quick test
      save_strategy="no",              # don't save for quick test
      remove_unused_columns=False,
  )

  # Quick classifier test
quick_classifier = Classifier(
      classifier="cell",
      cell_state_dict={"state_key": "disease_group", "states": "all"},
      max_ncells=100,                  # use only 100 cells
      training_args=quick_training_args.to_dict(),
      freeze_layers=8,                 # freeze more layers for speed
      num_crossval_splits=1,           # no cross-validation
      forward_batch_size=8,
      nproc=2,
      ngpu=1
  )

os.makedirs("quick_test_prepared", exist_ok=True)
os.makedirs("quick_test_results", exist_ok=True)

  # Quick preparation and validation
quick_classifier.prepare_data(
      input_data_file=subset_path,
      output_directory="quick_test_prepared",
      output_prefix="quick"
      )

In [6]:
quick_metrics = quick_classifier.validate(
        model_directory="./Geneformer/gf-12L-95M-i4096",
        prepared_input_data_file="quick_test_prepared/quick_labeled_train.dataset",
        id_class_dict_file="quick_test_prepared/quick_id_class_dict.pkl",
        output_directory="quick_test_results",
        output_prefix="quick_classifier",
        n_hyperopt_trials=0,             # no hyperopt for quick test
    )
    
print("Quick test completed!")
print(f"Quick test metrics: {quick_metrics}")

mkdir: cannot create directory ‘quick_test_results/250613_geneformer_cellClassifier_quick_classifier/’: File exists
  0%|          | 0/1 [00:00<?, ?it/s]mkdir: cannot create directory ‘quick_test_results/250613_geneformer_cellClassifier_quick_classifier/ksplit1’: File exists
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./Geneformer/gf-12L-95M-i4096 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/transformers/training_args.py:2029: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


****** Validation split: 1/1 ******



Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: Currently logged in as: tusharagashe1 (tusharagashe1-university-of-california) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/c4/home/tagashe/jdmwork/Geneformer/geneformer/collator_for_classification.py:668: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,1.385700,1.327843,0.500000,0.166667


Trainer.tokenizer is now deprecated. You should use `Trainer.processing_class = processing_class` instead.
  0%|          | 0/1 [00:14<?, ?it/s]


ValueError: index can't contain negative values